In [13]:
import sqlite3
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor


Data Preprocessing Steps

In [14]:
sensor_chosen = 16271
query = f"""
SELECT * FROM data_table
where sensor_index ={sensor_chosen}
"""

with sqlite3.connect('../datasets/dallas.sqlite') as db:
    data_chosen_sensor = pd.read_sql(query, db)


data_chosen_sensor['time_stamp'] = pd.to_datetime(data_chosen_sensor['time_stamp'])
data_chosen_sensor.set_index('time_stamp', inplace=True)
data_chosen_sensor =  data_chosen_sensor.sort_index(ascending=True)

# Merging weather data from noaa to the data from a single sensor
weather_noaa_data = pd.read_csv('../datasets/Dallas_stations_data.csv')


# choose one station
# USW00003971
# removed attributes columns and some others
weather_noaa_data_w =  weather_noaa_data[weather_noaa_data['STATION']=='USW00003971'][['STATION', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'DATE', 'AWND', 'DAPR', 'MDPR', 'PGTM', 'PRCP', 'SNOW', 'SNWD', 'TAVG','TMAX','TMIN', 'WDF2','WDF5', 'WESD', 'WESF','WSF2', 'WSF5', 'WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06','WT08']]

weather_noaa_data_w_n = weather_noaa_data_w[:-2] # the noaa data is for two days longer

weather_noaa_data_w_n = weather_noaa_data_w_n.fillna(0) # lot's of NAN values, which are essentially zeros

weather_noaa_data_w_n['DATE'] = pd.to_datetime(weather_noaa_data_w_n['DATE'])
weather_noaa_data_w_n.set_index('DATE', inplace=True) # setting index makes it easier to merge later

data_chosen_sensor.index = data_chosen_sensor.index.date # change index to date format

merged_df = pd.merge(data_chosen_sensor, weather_noaa_data_w_n, left_index=True, right_index=True)

#create time lag feature for pm2.5_atm_a data
merged_df['pm2.5_lag1'] = merged_df['pm2.5_atm_a'].shift(1)
merged_df = merged_df.fillna(0) # because the first entry is nan 

# Removing unwanted columns
columns_to_keep = [col for col in merged_df.columns if not col.endswith('_b') and col not in ['pm2.5_cf_1_a', 'STATION','LATITUDE','LONGITUDE','ELEVATION']]

merged_df = merged_df[columns_to_keep]

In [15]:
# check the merged dataframe
print(merged_df.head())

            sensor_index  humidity_a  temperature_a  pressure_a  pm2.5_atm_a  \
2022-04-01         16271      32.768         69.131     993.196        4.501   
2022-04-02         16271      44.143         74.520     993.565       11.086   
2022-04-03         16271      40.664         78.053     992.773       15.981   
2022-04-04         16271      44.896         77.997     987.005       11.059   
2022-04-05         16271      53.601         81.234     981.140       11.629   

            AWND  DAPR  MDPR  PGTM  PRCP  ...  WSF2  WSF5  WT01  WT02  WT03  \
2022-04-01   4.9   0.0   0.0   0.0   1.0  ...  11.2  13.9   0.0   0.0   0.0   
2022-04-02   3.1   0.0   0.0   0.0   1.3  ...   7.2  10.3   0.0   0.0   0.0   
2022-04-03   5.5   0.0   0.0   0.0   0.0  ...  10.7  15.2   0.0   0.0   0.0   
2022-04-04   5.4   0.0   0.0   0.0  26.2  ...  13.0  16.5   1.0   0.0   1.0   
2022-04-05   4.6   0.0   0.0   0.0   0.3  ...   9.4  14.3   1.0   0.0   0.0   

            WT04  WT05  WT06  WT08  pm2.5_la

In [16]:
# Handle missing values using SimpleImputer
imputer = SimpleImputer(strategy='mean')
df_imputed = pd.DataFrame(imputer.fit_transform(merged_df), columns=merged_df.columns, index=merged_df.index)

# Define the time period for testing
test_period = '2024-03-25'  # test period starts from this date and goes to the end of data

# Split the data into training and testing sets
train_data = df_imputed[:test_period]
test_data = df_imputed[test_period:]

# Separate features and target for both training and testing sets
X_train = train_data.drop(columns=['pm2.5_atm_a'])
y_train = train_data['pm2.5_atm_a']
X_test = test_data.drop(columns=['pm2.5_atm_a'])
y_test = test_data['pm2.5_atm_a']

print("Training Data:\n", train_data.head())
print("Testing Data:\n", test_data.head())


Training Data:
             sensor_index  humidity_a  temperature_a  pressure_a  pm2.5_atm_a  \
2022-04-01       16271.0      32.768         69.131     993.196        4.501   
2022-04-02       16271.0      44.143         74.520     993.565       11.086   
2022-04-03       16271.0      40.664         78.053     992.773       15.981   
2022-04-04       16271.0      44.896         77.997     987.005       11.059   
2022-04-05       16271.0      53.601         81.234     981.140       11.629   

            AWND  DAPR  MDPR  PGTM  PRCP  ...  WSF2  WSF5  WT01  WT02  WT03  \
2022-04-01   4.9   0.0   0.0   0.0   1.0  ...  11.2  13.9   0.0   0.0   0.0   
2022-04-02   3.1   0.0   0.0   0.0   1.3  ...   7.2  10.3   0.0   0.0   0.0   
2022-04-03   5.5   0.0   0.0   0.0   0.0  ...  10.7  15.2   0.0   0.0   0.0   
2022-04-04   5.4   0.0   0.0   0.0  26.2  ...  13.0  16.5   1.0   0.0   1.0   
2022-04-05   4.6   0.0   0.0   0.0   0.3  ...   9.4  14.3   1.0   0.0   0.0   

            WT04  WT05  WT06

Fit the temporal model 

In [17]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize and train the model
# model = RandomForestRegressor(n_estimators=100, random_state=42)
model = XGBRegressor()
model.fit(X_train_scaled, y_train)

# Make predictions
y_pred = model.predict(X_test_scaled)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Display a sample of actual vs predicted values
results = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print(results.head())
feature_importances = model.feature_importances_
features = X_train.columns

importance_df = pd.DataFrame({'Feature': features, 'Importance': feature_importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

print("\nFeature Importances:")
print(importance_df)

Mean Squared Error: 9.075900799476889
            Actual  Predicted
2024-03-25   5.214   5.272196
2024-03-26   2.510   2.560776
2024-03-27   4.769   7.504360
2024-03-28   6.219   5.702638
2024-03-29   3.781   7.670517

Feature Importances:
          Feature  Importance
26           WT08    0.225356
27     pm2.5_lag1    0.221356
22           WT03    0.058499
20           WT01    0.057127
13           TMIN    0.057048
8            PRCP    0.052195
12           TMAX    0.047942
14           WDF2    0.043356
18           WSF2    0.038736
15           WDF5    0.035622
21           WT02    0.033932
1      humidity_a    0.033327
4            AWND    0.030241
3      pressure_a    0.024990
2   temperature_a    0.022764
19           WSF5    0.017510
25           WT06    0.000000
24           WT05    0.000000
23           WT04    0.000000
7            PGTM    0.000000
9            SNOW    0.000000
17           WESF    0.000000
16           WESD    0.000000
5            DAPR    0.000000
6         

spatial model

In [44]:
query = """
SELECT 
    s.sensor_index, 
    s.name, 
    s.latitude, 
    s.longitude, 
    d.time_stamp,
    d.humidity_a, 
    d.temperature_a, 
    d.pressure_a, 
    d."pm2.5_atm_a", 
    d."pm2.5_atm_b", 
    d."pm2.5_cf_1_a", 
    d."pm2.5_cf_1_b"
FROM 
    sensor_table AS s
JOIN 
    data_table AS d
ON 
    s.sensor_index = d.sensor_index
WHERE 
    d.time_stamp ='2024-03-29T00:00:00Z'
    
"""

with sqlite3.connect('../datasets/dallas.sqlite') as db:
    data_d = pd.read_sql(query, db)

In [45]:
data_d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   sensor_index   64 non-null     int64  
 1   name           64 non-null     object 
 2   latitude       64 non-null     float64
 3   longitude      64 non-null     float64
 4   time_stamp     64 non-null     object 
 5   humidity_a     60 non-null     float64
 6   temperature_a  60 non-null     float64
 7   pressure_a     60 non-null     float64
 8   pm2.5_atm_a    64 non-null     float64
 9   pm2.5_atm_b    64 non-null     float64
 10  pm2.5_cf_1_a   64 non-null     float64
 11  pm2.5_cf_1_b   64 non-null     float64
dtypes: float64(9), int64(1), object(2)
memory usage: 6.1+ KB


In [46]:
coordinates = np.column_stack((data_d['latitude'], data_d['longitude']))

nn_model = NearestNeighbors(n_neighbors=5+1, algorithm='kd_tree')
nn_model.fit(coordinates)

distances, neighbors = nn_model.kneighbors(coordinates)
neighbors = neighbors[:, 1:]

data_d= data_d.set_index('sensor_index')
data_d.index[neighbors[0]].to_list()
data_d['nearest_neighbors'] = [data_d.index[neighbors[i]].to_list() for i in range(len(data_d))]
distances = distances[:,1:]
weights = 1/distances


weights_dict = {}
for i, sensor_idx in enumerate(data_d.index):
    neighbor_indices = data_d.index[neighbors[i]]
    weights_dict[sensor_idx] = pd.Series(weights[i], index=neighbor_indices)


spatial_weights = pd.DataFrame(weights_dict).fillna(0).T

spatial_weights = spatial_weights.reindex(index=data_d.index, columns=data_d.index, fill_value=0)

variables_of_interest = ['pm2.5_atm_a']

for var in variables_of_interest:
    feature_vector = data_d[var].values
    spatial_lag_vector = spatial_weights.values @ feature_vector
    data_d[f'spatial_lag_{var}'] = spatial_lag_vector

In [47]:
data_d.head()

,name,latitude,longitude,time_stamp,humidity_a,temperature_a,pressure_a,pm2.5_atm_a,pm2.5_atm_b,pm2.5_cf_1_a,pm2.5_cf_1_b,nearest_neighbors,spatial_lag_pm2.5_atm_a
sensor_index,,,,,,,,,,,,,
2644,Sachse Farms,32.990840,-96.599300,2024-03-29T00:00:00Z,51.843,72.680,995.137,4.294,4.859,4.294,4.993,"[87721, 196323, 144032, 196421, 147038]",261.195258
12969,Meadow Glen,32.991665,-96.859150,2024-03-29T00:00:00Z,59.757,76.795,993.019,3.432,4.307,3.432,4.307,"[113833, 147038, 113642, 59953, 196421]",165.849581
13013,GPTX,32.766370,-97.037926,2024-03-29T00:00:00Z,44.971,75.011,999.182,4.360,4.214,4.360,4.214,"[135002, 16271, 184053, 99595, 109566]",102.591781
16271,Arbormont Estates,32.882004,-97.084130,2024-03-29T00:00:00Z,50.543,73.336,993.643,3.781,3.064,3.781,3.064,"[184053, 135002, 46221, 13013, 127049]",156.304867
46221,Thornbridge East,32.893590,-97.190370,2024-03-29T00:00:00Z,46.126,72.078,992.357,2.744,3.137,2.744,3.137,"[51821, 184053, 16271, 99585, 127045]",181.389618


In [48]:
def get_train_test_data_for_sensor(data, sensor_index):
    data = data[
        ~(data['pm2.5_atm_a'] > data['pm2.5_atm_b'] + 10) &
        ~(data['pm2.5_atm_a'] < data['pm2.5_atm_b'] - 10)
    ]
    train_data = data[data.index != sensor_index]
    test_data = data[data.index == sensor_index]
    
    X_train = train_data[['humidity_a', 'temperature_a', 'pressure_a', 'spatial_lag_pm2.5_atm_a']]
    y_train = train_data['pm2.5_atm_a']
    X_test = test_data[['humidity_a', 'temperature_a', 'pressure_a', 'spatial_lag_pm2.5_atm_a']]
    y_test = test_data['pm2.5_atm_a']
    
    return X_train, X_test, y_train, y_test

specific_sensor_index = 16271

X_train, X_test, y_train, y_test = get_train_test_data_for_sensor(data_d, specific_sensor_index)

print("Training Data:\n", X_train.head())
print("Testing Data:\n", X_test.head())


Training Data:
               humidity_a  temperature_a  pressure_a  spatial_lag_pm2.5_atm_a
sensor_index                                                                
2644              51.843         72.680     995.137               261.195258
12969             59.757         76.795     993.019               165.849581
13013             44.971         75.011     999.182               102.591781
46221             46.126         72.078     992.357               181.389618
51821             47.567         74.078     992.738               146.321441
Testing Data:
               humidity_a  temperature_a  pressure_a  spatial_lag_pm2.5_atm_a
sensor_index                                                                
16271             50.543         73.336     993.643               156.304867


In [49]:
def train_evaluate_xgboost(X_train, X_test, y_train, y_test):
    # Define pipeline with SimpleImputer and XGBRegressor
    pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('xgbreg', XGBRegressor(n_estimators=100, random_state=42))
    ])

    # Train XGBoost model with pipeline
    pipeline.fit(X_train, y_train)

    # Predict PM2.5 values
    y_pred = pipeline.predict(X_test)
    
    # Calculate evaluation metrics
    mae = mean_absolute_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    r2 = r2_score(y_test, y_pred)
    
    # Print the evaluation metrics
    print("XGBoost Regression")
    print("MAE:", mae)
    print("RMSE:", rmse)
    print("R2 Score:", r2)
    
    return y_pred

In [50]:
train_evaluate_xgboost(X_train, X_test, y_train, y_test)

XGBoost Regression
MAE: 2.157262903213501
RMSE: 2.157262903213501
R2 Score: nan


/home/prahlad/anaconda3/envs/pyml/lib/python3.9/site-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


array([1.6237371], dtype=float32)

In [25]:
y_test

sensor_index
16271    3.781
Name: pm2.5_atm_a, dtype: float64

In [53]:
start_date = '2024-03-25'
end_date = '2024-03-29'

# Define the SQL query with the date range
query = f"""
SELECT 
    s.sensor_index, 
    s.name, 
    s.latitude, 
    s.longitude, 
    d.time_stamp,
    d.humidity_a, 
    d.temperature_a, 
    d.pressure_a, 
    d."pm2.5_atm_a", 
    d."pm2.5_atm_b", 
    d."pm2.5_cf_1_a", 
    d."pm2.5_cf_1_b"
FROM 
    sensor_table AS s
JOIN 
    data_table AS d
ON 
    s.sensor_index = d.sensor_index
WHERE 
    d.time_stamp BETWEEN '{start_date}T00:00:00Z' AND '{end_date}T23:59:59Z'
"""

# Fetch data from the SQLite database
with sqlite3.connect('../datasets/dallas.sqlite') as db:
    data_d = pd.read_sql(query, db)

In [54]:
data_d['time_stamp'] = pd.to_datetime(data_d['time_stamp'])

In [55]:
data_d.set_index('time_stamp', inplace=True)

In [56]:
data_d.index = data_d.index.date

In [57]:
data_d = data_d.sort_index(ascending=False)

In [58]:
data_d.head()

,sensor_index,name,latitude,longitude,humidity_a,temperature_a,pressure_a,pm2.5_atm_a,pm2.5_atm_b,pm2.5_cf_1_a,pm2.5_cf_1_b
2024-03-29,2644,Sachse Farms,32.990840,-96.599300,51.843,72.680,995.137,4.294,4.859,4.294,4.993
2024-03-29,114329,UNT-GEO-66,33.159290,-97.137160,47.057,71.616,994.884,0.883,1.257,0.883,1.257
2024-03-29,113975,UNT-GEO-37,33.141315,-97.035835,41.245,74.676,995.113,0.685,0.446,0.685,0.446
2024-03-29,113969,UNT-GEO-58,33.114075,-97.365974,75.111,68.013,988.733,0.845,3298.304,0.845,4948.424
2024-03-29,113857,UNT-GEO-13024,33.133965,-96.765780,45.909,70.293,988.840,0.651,1.958,0.651,1.958


In [59]:
def get_train_test_data_for_sensor_(data, sensor_index):
    train_data = data[data['sensor_index']!= sensor_index]
    test_data = data[data['sensor_index'] == sensor_index]
    
    X_train = train_data[['humidity_a', 'temperature_a', 'pressure_a', 'spatial_lag_pm2.5_atm_a']]
    y_train = train_data['pm2.5_atm_a']
    X_test = test_data[['humidity_a', 'temperature_a', 'pressure_a', 'spatial_lag_pm2.5_atm_a']]
    y_test = test_data['pm2.5_atm_a']
    
    return X_train, X_test, y_train, y_test

In [69]:
def train_evaluate_xgboost_(X_train, X_test, y_train, y_test):
    # Define pipeline with SimpleImputer and XGBRegressor
    pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('xgbreg', XGBRegressor(n_estimators=100, random_state=42))
    ])

    # Train XGBoost model with pipeline
    pipeline.fit(X_train, y_train)

    # Predict PM2.5 values
    y_pred = pipeline.predict(X_test)
    
    # Calculate evaluation metrics
    mae = mean_absolute_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    r2 = r2_score(y_test, y_pred)
    
    return y_test, mae, rmse, r2, y_pred

In [70]:
specific_sensor_index = 16271
y_predictions = []
for sensor_index in data_d.index.unique():
    sensor_data = data_d[data_d.index==sensor_index]
    variables_of_interest = ['pm2.5_atm_a']
    for var in variables_of_interest:
        feature_vector = sensor_data[var].values
        spatial_lag_vector = spatial_weights.values @ feature_vector
        sensor_data[f'spatial_lag_{var}'] = spatial_lag_vector
    sensor_data = sensor_data[
        ~(sensor_data['pm2.5_atm_a'] > sensor_data['pm2.5_atm_b'] + 10) &
        ~(sensor_data['pm2.5_atm_a'] < sensor_data['pm2.5_atm_b'] - 10)
    ]
    
    X_train, X_test, y_train, y_test = get_train_test_data_for_sensor_(sensor_data, specific_sensor_index)
    y_predictions.append(train_evaluate_xgboost_(X_train, X_test, y_train, y_test))
    

/tmp/ipykernel_7355/2470860228.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sensor_data[f'spatial_lag_{var}'] = spatial_lag_vector
/home/prahlad/anaconda3/envs/pyml/lib/python3.9/site-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/tmp/ipykernel_7355/2470860228.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sensor_data[f'spatial_lag_{var}'] = spatial_lag_vector
/home/prahl

In [71]:
y_predictions

[(2024-03-29    3.781
  Name: pm2.5_atm_a, dtype: float64,
  1.957759958267212,
  1.957759958267212,
  nan,
  array([1.82324], dtype=float32)),
 (2024-03-28    6.219
  Name: pm2.5_atm_a, dtype: float64,
  1.1729823532104495,
  1.1729823532104495,
  nan,
  array([5.0460176], dtype=float32)),
 (2024-03-27    4.769
  Name: pm2.5_atm_a, dtype: float64,
  0.6930756034851075,
  0.6930756034851075,
  nan,
  array([4.0759244], dtype=float32)),
 (2024-03-26    2.51
  Name: pm2.5_atm_a, dtype: float64,
  3.279604663848877,
  3.279604663848877,
  nan,
  array([5.7896047], dtype=float32)),
 (2024-03-25    5.214
  Name: pm2.5_atm_a, dtype: float64,
  1.0069701538085933,
  1.0069701538085933,
  nan,
  array([6.22097], dtype=float32))]

Date            Actual      Predicted

2024-03-25      5.214     5.272196

2024-03-26      2.510     2.560776

2024-03-27      4.769      7.504360

2024-03-28      6.219       5.702638

2024-03-29      3.781      7.670517